In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import coremltools as ct
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'pandas'

Load both new CSV file and old CSV file. I will make sure the app send csv files with equal number of rows  
and making sure the first column is label

In [ ]:
def load_and_preprocess_data(new_csv_file, old_files_directory):
    new_data = pd.read_csv(new_csv_file)
    new_labels = new_data.iloc[:, 0].values
    new_features = new_data.iloc[:, 1:].values

    all_features = [new_features]
    all_labels = [new_labels]

    for file in os.listdir(old_files_directory):
        if file.endswith(".csv"):
            old_data = pd.read_csv(os.path.join(old_files_directory, file))
            all_labels.append(old_data.iloc[:, 0].values)
            all_features.append(old_data.iloc[:, 1:].values)

    combined_features = np.vstack(all_features)
    combined_labels = np.hstack(all_labels)

# Not sure if the data is already normalized, but I'll normalize it just in case
    scaler = StandardScaler()
    combined_features = scaler.fit_transform(combined_features)

    return combined_features, combined_labels

Preprocess data (We should need an inpute process in case some data is missing) etc.

I'm using Backward Propagation neural network. https://www.geeksforgeeks.org/backpropagation-in-neural-network/

In [ ]:
def main(new_csv_file, old_files_directory):
    features, labels = load_and_preprocess_data(new_csv_file, old_files_directory)
    # I will make sure the labels are integers before sending them to the server, just in case here.
    labels = labels.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=69)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    input_size = X_train.shape[1]
    output_size = len(np.unique(labels))

    model = nn.Sequential(
        nn.Linear(input_size, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, output_size)
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    num_epochs = 69
    for epoch in range(num_epochs):
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred = model(X_test)
        y_pred_classes = torch.argmax(y_pred, axis=1)
        acc = (y_pred_classes == y_test).float().mean()
        # We dont need this in actual deployment
        print(f"Test Accuracy: {acc:.4f}")

    # Below is how to send the model to CoreML, havent figure out yet